In [0]:
!pip install pytorch-transformers
!pip install tensorboardX

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from __future__ import absolute_import, division, print_function

from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from tensorboardX import SummaryWriter
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset, ConcatDataset, Subset)
from pytorch_transformers import AdamW, WarmupLinearSchedule
import csv
from google.colab import drive
import glob
import logging
import os
import sys
from io import open
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
import numpy as np
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix
from scipy.stats import pearsonr

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
csv.field_size_limit(2147483647)

2147483647

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path = '/content/drive/My Drive/thesis_practical/bert_classification'
model_name = 'bert-base-cased'
model_type =  'bert'
output_dir = 'outputs_hf/'
cache_dir = 'cache_hf/'
fp16_opt_level = 'O1'
max_seq_length = 128
train_batch_size = 32
eval_batch_size = 32
gradient_accumulation_steps = 1
num_train_epochs = 5
weight_decay = 0
learning_rate = 4e-5
adam_epsilon = 1e-8
warmup_steps = 0
max_grad_norm = 1.0

overwrite_output_dir = True
reprocess_input_data = True
logging_steps = 50
evaluate_during_training = False
save_steps = 2000
eval_all_checkpoints = True

train_file = 'train.tsv'
test_file = 'test.tsv'
unlabeled_file = 'unlabeled.tsv'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [0]:
with open(f'{path}/data/dataset_preprocessed.tsv', 'r') as f:
  lines = f.readlines()
random.shuffle(lines)
train_end = int(len(lines)*0.8)
# for line in lines:
  # print(line)
train_lines = lines[:train_end]
test_lines = lines[train_end:]
with open(f'{path}/data/train_preproc.tsv', 'w') as f:
  for item in train_lines:
    f.write("%s" % item)
with open(f'{path}/data/test_preproc.tsv', 'w') as f:
  for item in test_lines:
    f.write("%s" % item)

In [0]:
dataset = pd.read_csv(f'{path}/data/unlabeled_dataset/unlabeled_sentences.csv', sep=',', error_bad_lines=False, header=None)
#print(dataset[0])
#print(dataset[1])
train_df, test_df, train_y, test_y = train_test_split(dataset[0], dataset[1], test_size=0.2)
print(train_df)
print(train_y)
print(train_df.shape[0])

KeyError: ignored

In [0]:
train_df = pd.read_csv(f'{path}/data/unlabeled_dataset/unlabeled_sentences.csv', header=None)
# test_df = pd.read_csv(f'{dataset_path}/test.csv', header=None)
# print(train_df[0])
# print(train_df[1])

#train_df = list(train_df)
print(train_df[0])
# train_y = list(train_y)
# test_df = list(test_df)
# test_y = list(test_y)
train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':'0',
    'alpha':['a']*train_df.shape[0],
    'text': train_df[0].replace(r'\n', ' ', regex=True)
})

# test_df_bert = pd.DataFrame({
#     'id':range(len(test_df)),
#     'label':test_y,
#     'alpha':['a']*test_df.shape[0],
#     'text': test_df.replace(r'\n', ' ', regex=True)
# })

0    At the same time. one does not want to lose th...
1    It will require the use of service and/or remo...
2    if users can be on standby at a suitably equip...
3    However taking into account that the Watcom GU...
4    Since the GUI library relies on OS/2 PM and Wi...
5    It will be an additional work for the programm...
6    The person, or persons, who pay for the produc...
7    In the context of this recommended practice th...
8    The person, or persons, who produce a product ...
9    In the context of this recommended practice, t...
Name: 0, dtype: object


In [0]:
train_df_bert.to_csv(f'{path}/data/unlabeled_dataset/unlabeled.tsv', sep='\t', index=False, header=False)
#test_df_bert.to_csv(f'{path}/data/test.tsv', sep='\t', index=False, header=False)

In [0]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample."""
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_unlabeled_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the unlabeled set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, train_file)), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, test_file)), "test")
        
    def get_unlabeled_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, unlabeled_file)), "unlabeled")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


def convert_example_to_feature(example_row, pad_token=0, sequence_a_segment_id=0, sequence_b_segment_id=1,
                                cls_token_segment_id=1, pad_token_segment_id=0, mask_padding_with_zero=True):
    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, \
        cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)

In [0]:
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True):
    """ Loads a data file into a list of `InputBatch`s """

    label_map = {label : i for i, label in enumerate(label_list)}

    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id) for example in examples]

    process_count = cpu_count() - 2
    if (process_count < 1):
      process_count = 1
    print(process_count)
    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=100), total=len(examples)))


    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maxim um length."""

    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [0]:
processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

GLUE_TASKS_NUM_LABELS = {
    "binary": 2
}

In [0]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]

In [0]:
processor = processors['binary']()
label_list = processor.get_labels()
num_labels = len(label_list)

In [0]:
config = config_class.from_pretrained(model_name, num_labels=2, finetuning_task='binary')
tokenizer = tokenizer_class.from_pretrained(model_name)
model = model_class.from_pretrained(model_name)
model.to(device);

task = 'binary'

processor = processors['binary']()
label_list = processor.get_labels()
num_labels = len(label_list)

INFO:pytorch_transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpfkhbqde0





100%|██████████| 361/361 [00:00<00:00, 232017.12B/s]
INFO:pytorch_transformers.file_utils:copying /tmp/tmpfkhbqde0 to cache at /root/.cache/torch/pytorch_transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.3d5adf10d3445c36ce131f4c6416aa62e9b58e1af56b97664773f4858a46286e
INFO:pytorch_transformers.file_utils:creating metadata file for /root/.cache/torch/pytorch_transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.3d5adf10d3445c36ce131f4c6416aa62e9b58e1af56b97664773f4858a46286e
INFO:pytorch_transformers.file_utils:removing temp file /tmp/tmpfkhbqde0
INFO:pytorch_transformers.modeling_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /root/.cache/torch/pytorch_tra

In [0]:
def load_and_cache_examples(task, tokenizer, evaluate=False, unlabeled=False):
    processor = processors[task]()
    output_mode = 'classification'

    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(f'{path}/', f"cached_{mode}_{model_name}_{max_seq_length}_{task}")

    if os.path.exists(cached_features_file) and not reprocess_input_data:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)       
    else:
        logger.info("Creating features from dataset file at %s", f'{path}/data/')
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(f'{path}/data/active_active_datasetdataset/') if evaluate else processor.get_train_examples(f'{path}/data/active_dataset/') if not unlabeled else processor.get_unlabeled_examples(f'{path}/data/unlabeled_dataset/')
        print(examples)
        print(f'Length of examples: {len(examples)}')
        features = convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_mode, \
          cls_token_at_end=bool(model_type in ['xlnet']), \
          cls_token=tokenizer.cls_token, \
          sep_token=tokenizer.sep_token, \
          cls_token_segment_id=2 if model_type in ['xlnet'] else 0, \
          pad_on_left=bool(model_type in ['xlnet']), \
          pad_token_segment_id=4 if model_type in ['xlnet'] else 0)
        
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)
        
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset, examples

In [0]:
def create_dataloader(dataset, unlabeled_dataset=None, batch_size=64):
    train_sampler = RandomSampler(dataset)
    train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)
    unlabeled_dataloader =  None
    if unlabeled_dataset is not None:
        unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size=batch_size, shuffle=False)
    
    return train_dataloader, unlabeled_dataloader

def train(train_dataset, model, tokenizer, unlabeled_dataset=None):
    tb_writer = SummaryWriter()

    train_dataloader, unlabeled_dataloader = create_dataloader(train_dataset, unlabeled_dataset, train_batch_size)
    
    t_total = len(train_dataloader) // gradient_accumulation_steps * num_train_epochs
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_steps, t_total=t_total)
    
    # try:
    #     from apex import amp
    # except ImportError:
    #     raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    # model, optimizer = amp.initialize(model, optimizer, opt_level=fp16_opt_level)
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", num_train_epochs)
    logger.info("  Total train batch size  = %d", train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(num_train_epochs), desc="Epoch")
    
    for _ in train_iterator:
        global_step, tr_loss, logging_loss, train_dataloader, unlabeled_dataloader, \
          train_dataset, unlabeled_dataset \
          = train_epoch(model, global_step, tr_loss, logging_loss, tb_writer, \
                        tokenizer, optimizer, scheduler, train_dataset, unlabeled_dataset, \
                        train_dataloader, unlabeled_dataloader)


    return global_step, tr_loss / global_step, 

def train_epoch(model, global_step, tr_loss, logging_loss, tb_writer, tokenizer, optimizer, scheduler, train_dataset, unlabeled_dataset, train_dataloader, unlabeled_dataloader):
    epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
    for step, batch in enumerate(epoch_iterator):
        # -------------------------!!!!!!!!!!!!!!!!!!!!-------------------
        if (step == 10):
            break
        model.train()
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2] if model_type in ['bert', 'xlnet'] else None, 
                  'labels':         batch[3]}
        outputs = model(**inputs)
        loss = outputs[0]
        print("\r%f" % loss, end='')

        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        # if fp16:
        #     with amp.scale_loss(loss, optimizer) as scaled_loss:
        #         scaled_loss.backward()
        #     torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
            
        # else:
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        tr_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step() 
            
            model.zero_grad()
            global_step += 1

            if logging_steps > 0 and global_step % logging_steps == 0:
                # Log metrics
                if evaluate_during_training:  # Only evaluate when single GPU otherwise metrics may not average well
                    results = evaluate(model, tokenizer)
                    for key, value in results.items():
                        tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                tb_writer.add_scalar('loss', (tr_loss - logging_loss)/logging_steps, global_step)
                logging_loss = tr_loss

            if save_steps > 0 and global_step % save_steps == 0:
                # Save model checkpoint
                output_dir = os.path.join(f'{path}/{output_dir}', 'checkpoint-{}'.format(global_step))
                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)
                model_to_save = model.module if hasattr(model, 'module') else model  
                model_to_save.save_pretrained(output_dir)
                logger.info("Saving model checkpoint to %s", output_dir)

    # Active learning
    preds = None
    if unlabeled_dataset is not None and unlabeled_dataset.__len__() > 0:
        epoch_iterator = tqdm_notebook(unlabeled_dataloader, desc="Iteration (unlabeled)")
        for step, batch in enumerate(epoch_iterator):
            model.eval()
            batch = tuple(t.to(device) for t in batch)

            with torch.no_grad():
                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'token_type_ids': batch[2] if model_type in ['bert', 'xlnet'] else None,
                          'labels':         batch[3]}
                outputs = model(**inputs)
                tmp_eval_loss, logits = outputs[:2]
            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs['labels'].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

        uncertain_label_0 = np.any([preds[:,0] > -1, preds[:,0] < 1], axis=0)
        uncertain_label_1 = np.any([preds[:,1] > -1, preds[:,1] < 1], axis=0)
        equal_label_pred = np.any([abs(preds[:,0] - preds[:,1]) >= 0, abs(preds[:,0] - preds[:,1]) < 2], axis=0)
        # 'True' if need labling
        candidates_for_labling = np.any([uncertain_label_0, uncertain_label_1, equal_label_pred], axis=0)
        idx_of_candidates = np.argwhere(candidates_for_labling == True).flatten()

        active_labeled = []

        for (idx, i) in enumerate(idx_of_candidates):
            print(f'--- {idx + 1} of {len(idx_of_candidates)}')
            print(f'{unlabeled_examples[idx].text_a}')
            label = None
            while(label != 0 and label != 1 and label != -1):
                try:
                    if (label is None):
                      label = int(input('* Is it Software Requrement?: '))
                    else:
                      label = int(input('* Incorrect input. Enter \'1\' for true, \'0\' for false, \'-1\' to stop labling : '))
                except ValueError:
                    label = 'wrong'
                
            if label == -1:
                print('Stop labling')
                break
            # After sentence was labeled
            active_labeled.append((idx, label))

        # Update datasets
        train_dataset, unlabeled_dataset = update_dataset(train_dataset, unlabeled_dataset, active_labeled)
        train_dataloader, unlabeled_dataloader = create_dataloader(train_dataset, unlabeled_dataset, train_batch_size)

    return global_step, tr_loss, logging_loss, train_dataloader, unlabeled_dataloader, train_dataset, unlabeled_dataset 
    
def update_dataset(dataset, unlabeled_dataset, active_labeled):
    global unlabeled_examples
    labeled_dataset = [ dataset ]
    unlabeled_id = set(list(range(0, unlabeled_dataset.__len__())))
    for tuple in active_labeled:
        idx = tuple[0]
        label = tuple[1]
        unlabeled_id.remove(idx)
        unlabeled_dataset.__getitem__(idx)
        ds = TensorDataset(unlabeled_dataset.__getitem__(idx)[0].unsqueeze(0), unlabeled_dataset.__getitem__(idx)[1].unsqueeze(0),\
                                                                       unlabeled_dataset.__getitem__(idx)[2].unsqueeze(0),\
                                            torch.LongTensor([label]))
        labeled_dataset.append(ds)  
    dataset = ConcatDataset(labeled_dataset)
    #print(unlabeled_id)
    unlabeled_dataset = Subset(unlabeled_dataset, list(unlabeled_id))
    unlabeled_examples = [unlabeled_examples[i] for i in unlabeled_id]
    update_files(active_labeled)
    return dataset, unlabeled_dataset

def update_files(active_labeled):
    dataset_path = ''
    unlabeled_dataset_path = ''
    unlabeled_dataset = pd.read_csv(unlabeled_dataset_path, sep='\t', error_bad_lines=False, header=None, index_col=0)
    dataset = pd.read_csv(dataset_path, sep='\t', error_bad_lines=False, header=None, index_col=0)
    for tuple in active_labeled:
        idx = tuple[0]
        label = tuple[1]
        unlabeled_dataset.loc[idx, 0] = label
        dataset.append(unlabeled_dataset.loc[idx], ignore_index=True)
        unlabeled_dataset.drop(idx)
    unlabeled_dataset = ulabeled_dataset.reset_index(drop=True)

    unlabeled_dataset.to_csv(unlabeled_dataset_path, index=0, header=False, sep='\t')
    dataset.to_csv(dataset_path, index=0, header=False, sep='\t')
    print(f'Files {dataset_path} and {unlabeled_dataset_path} were updated')



In [0]:
def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(f'{path}/data/')
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    f1 = f1_score(labels, preds)
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "f1": f1
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels):
    assert len(preds) == len(labels)
    return get_eval_report(labels, preds)

def evaluate(model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = f'{path}/{output_dir}'

    results = {}
    EVAL_TASK = 'binary'

    eval_dataset, _ = load_and_cache_examples(EVAL_TASK, tokenizer, evaluate=True)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(dataset=eval_dataset, batch_size=eval_batch_size, \
                                 sampler=eval_sampler)

    # evaluation phase
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if model_type in ['bert', 'xlnet'] else None,
                      'labels':         batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    preds = np.argmax(preds, axis=1)

    result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids)
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        logger.info(f"Model type: {model_type}, model name: {model_name}")
        logger.info(f"Max seq len: {max_seq_length}, train batch size: {train_batch_size}")
        logger.info(f"Weight decay: {weight_decay}, lr: {learning_rate}, adam epsilon: {adam_epsilon}")
        logger.info(f"warmup steps: {warmup_steps}, max grad norm: {max_grad_norm}, gradient accumulation steps: {gradient_accumulation_steps})")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results, wrong

In [0]:
train_dataset, _ = load_and_cache_examples(task, tokenizer)
unlabeled_dataset, unlabeled_examples = load_and_cache_examples(task, tokenizer, unlabeled=True)
global_step, tr_loss = train(train_dataset, model, tokenizer, unlabeled_dataset)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

INFO:__main__:Creating features from dataset file at /content/drive/My Drive/thesis_practical/bert_classification/data/


[<__main__.InputExample object at 0x7f34a4950518>, <__main__.InputExample object at 0x7f34a49507f0>, <__main__.InputExample object at 0x7f34a37eb160>, <__main__.InputExample object at 0x7f34a37eb1d0>, <__main__.InputExample object at 0x7f34a37eb240>, <__main__.InputExample object at 0x7f34a37eb2b0>, <__main__.InputExample object at 0x7f34a37eb320>, <__main__.InputExample object at 0x7f34a37eb390>, <__main__.InputExample object at 0x7f34a37eb400>, <__main__.InputExample object at 0x7f34a37eb470>, <__main__.InputExample object at 0x7f34a37eb4a8>, <__main__.InputExample object at 0x7f34a37eb4e0>, <__main__.InputExample object at 0x7f34a37eb518>, <__main__.InputExample object at 0x7f34a37eb550>, <__main__.InputExample object at 0x7f34a37eb588>, <__main__.InputExample object at 0x7f34a37eb5c0>, <__main__.InputExample object at 0x7f34a37eb5f8>, <__main__.InputExample object at 0x7f34a37eb630>, <__main__.InputExample object at 0x7f34a37eb668>, <__main__.InputExample object at 0x7f34a37eb6a0>,

100%|██████████| 7387/7387 [00:02<00:00, 2981.43it/s]
INFO:__main__:Saving features into cached file /content/drive/My Drive/thesis_practical/bert_classification/cached_train_bert-base-cased_128_binary
INFO:__main__:Creating features from dataset file at /content/drive/My Drive/thesis_practical/bert_classification/data/


[<__main__.InputExample object at 0x7f34a3822128>, <__main__.InputExample object at 0x7f34a38220f0>, <__main__.InputExample object at 0x7f34a3822198>, <__main__.InputExample object at 0x7f34a38221d0>, <__main__.InputExample object at 0x7f34a3822208>, <__main__.InputExample object at 0x7f34a3822240>, <__main__.InputExample object at 0x7f34a3822278>, <__main__.InputExample object at 0x7f34a38222b0>, <__main__.InputExample object at 0x7f34a38222e8>, <__main__.InputExample object at 0x7f34a3822320>]
Length of examples: 10
2


100%|██████████| 10/10 [00:00<00:00, 354.03it/s]
INFO:__main__:Saving features into cached file /content/drive/My Drive/thesis_practical/bert_classification/cached_train_bert-base-cased_128_binary
INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 7387
INFO:__main__:  Num Epochs = 5
INFO:__main__:  Total train batch size  = 32
INFO:__main__:  Gradient Accumulation steps = 1
INFO:__main__:  Total optimization steps = 1155
Epoch:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

Epoch:  60%|██████    | 3/5 [02:33<01:56, 58.16s/it]

0.415640

Epoch:  80%|████████  | 4/5 [02:38<00:41, 41.98s/it]

0.604841

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Epoch: 100%|██████████| 5/5 [02:42<00:00, 32.49s/it]
INFO:__main__: global_step = 50, average loss = 0.5936674982309341


In [0]:
if not os.path.exists(f'{path}/{output_dir}'):
    os.makedirs(f'{path}/{output_dir}')
    logger.info("Saving model checkpoint to %s", f'{path}/{output_dir}')

model_to_save = model.module if hasattr(model, 'module') else model 
model_to_save.save_pretrained(f'{path}/{output_dir}')
tokenizer.save_pretrained(f'{path}/{output_dir}')

('/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf/vocab.txt',
 '/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf/special_tokens_map.json',
 '/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf/added_tokens.json')

In [0]:
checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(f'{path}/{output_dir}' + '/**/' + WEIGHTS_NAME, recursive=True)))
print(checkpoints[2:len(checkpoints)-1])

['/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf']


In [0]:
results = {}

checkpoints = [f'{path}/{output_dir}']
if eval_all_checkpoints:
    checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(f'{path}/{output_dir}' + '/**/' + WEIGHTS_NAME, recursive=True)))
    logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)
logger.info("Evaluate the following checkpoints: %s", checkpoints)
for checkpoint in checkpoints[2:len(checkpoints)-1]:
    global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
    model = model_class.from_pretrained(checkpoint)
    model.to(device)
    result, wrong_preds = evaluate(model, tokenizer, prefix=global_step)
    result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
    results.update(result)

INFO:__main__:Evaluate the following checkpoints: ['/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf/XLNet', '/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf/bert_cased', '/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf', '/content/drive/My Drive/thesis_practical/bert_classification/outputs_hf/roberto']
INFO:__main__:Creating features from dataset file at /content/drive/My Drive/thesis_practical/bert_classification/data/


Length of examples: 2019
2


100%|██████████| 2019/2019 [00:00<00:00, 2666.79it/s]
INFO:__main__:Saving features into cached file /content/drive/My Drive/thesis_practical/bert_classification/cached_dev_bert-base-uncased_128_binary
INFO:__main__:***** Running evaluation /content/drive/My Drive/thesis_practical/bert_classification/outputs_hf *****
INFO:__main__:  Num examples = 2019
INFO:__main__:  Batch size = 32


INFO:__main__:***** Eval results /content/drive/My Drive/thesis_practical/bert_classification/outputs_hf *****
INFO:__main__:Model type: bert, model name: bert-base-uncased
INFO:__main__:Max seq len: 128, train batch size: 32
INFO:__main__:Weight decay: 0, lr: 4e-05, adam epsilon: 1e-08
INFO:__main__:warmup steps: 0, max grad norm: 1.0, gradient accumulation steps: 1)
INFO:__main__:  f1 = 0.4638047138047138
INFO:__main__:  fn = 349
INFO:__main__:  fp = 925
INFO:__main__:  mcc = -0.24034528377010173
INFO:__main__:  tn = 194
INFO:__main__:  tp = 551


In [0]:
global_step = 0

def step(step):
    step = step + 1
    return step

step(global_step)
step(global_step)
print(global_step, step(global_step))

0 1
